# Data Processing Steps

explaination of the project .....

### Imports

In [1]:
from urllib.request import urlopen
from gensim.models import Word2Vec       
import gzip                                                                                                                                                  
import pandas as pd
import numpy as np                                                                                                                    

### Download structure chain Id from Pisces 
- sequence idendity 20%
- resolution 2.5

In [2]:
fileURL = "http://dunbrack.fccc.edu/Guoli/culledpdb_hh" + "/cullpdb_pc20_res2.5_R1.0_d180913_chains9046.gz"                                                  
u = urlopen(fileURL)                                                                                                                                         
line = str(gzip.GzipFile(fileobj=u).read()).split('\\n')                                                                                                     
pisces_structureChainId = [l.split()[0][:4] + '.' + l.split()[0][4] for l in line if len(l.split()) > 1][1:] # remove headings
 
# show 10 examples
print(pisces_structureChainId[:10])

['12AS.A', '16VP.A', '1A1X.A', '1A3C.A', '1A41.A', '1A5T.A', '1A62.A', '1A73.A', '1A7J.A', '1A92.B']


### Parse structure & sequence from fasta file   

In [10]:
# unzip the gziped file, convert byte string to string, remove bytes character and quotation mark and store result in a list
raw_text = [str(line)[2:-1] for line in gzip.open('./data/ss_dis.txt.gz','rb')]

# get line number of lines with structure chain ID (starts with '>)
line_numbers = [i for i,n in enumerate(raw_text) if n[0] == '>']

# get the structure chain id from the from every 3 lines in the list of line numbers (the same structurechainId repreats 3times)
structures = ['.'.join(raw_text[line].split(":")[:2])[1:] for line in line_numbers[::3]]   

# functino to join lines of string into a single string and remove new line characters
parse_input = lambda x: ''.join(x).replace("\\n","")

# data parsed into format: [structureChainId, amino acid sequence, secondary structure]
# missing secondary structure labeled as coil
structure_sequence = [[s,parse_input(raw_text[a+1:b]),parse_input(raw_text[b+1:c]).replace(' ','C')] for s,a,b,c in zip(structures, *[iter(line_numbers)]*3)]

# Show 1 example of parsed data
print(f"Number of prased in structure from fasta file: {len(structure_sequence)}")
print(structure_sequence[1])

Number of prased in structure from fasta file: 402007
['102L.A', 'MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTNSLRMLQQKRWDEAAVNLAKSRWYNQTPNRAKRVITTFRTGTWDAYKNL', 'CCHHHHHHHHHCCEEEEEECTTSCEEEETTEEEESSSCTTTHHHHHHHHHHTSCCTTBCCHHHHHHHHHHHHHHHHHHHHHCTTHHHHHHHSCHHHHHHHHHHHHHHHHHHHHTCHHHHHHHHTTCHHHHHHHHHSSHHHHHSHHHHHHHHHHHHHSSSGGGCCC']


### Filter structure sequence by pisces representative set

In [4]:
structure_sequence_filtered = [s for s in structure_sequence if s[0] in pisces_structureChainId]
print(f"Number of structure after filtering with pisces set: {len(structure_sequence_filtered)}")

Number of structure after filtering with pisces set: 8947


### Store results in pandas DataFrame

In [5]:
dic = {"structureChainId" : [s[0] for s in structure_sequence_filtered],                                                                                     
       "sequence" : [s[1] for s in structure_sequence_filtered],                                                                                             
       "secondary_structure" : [s[2] for s in structure_sequence_filtered]                                                                                   
       }                                                                                                                                                     
df = pd.DataFrame(dic)
df = df[["structureChainId", "sequence", "secondary_structure"]]
df.head(10)

,structureChainId,sequence,secondary_structure
0,12AS.A,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,CCCCHHHHHHHHHHHHHHHHHHHHHHHCEEECCCCSEEETTSSCSC...
1,16VP.A,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,CCSCCCCCCCCHHHHHHHHHHHHTCTTHHHHHHHHHHCCCCCSTTS...
2,1A1X.A,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,CCCCCCCCCCCSEEEEEETTEEEETTSCEEEEEEEECSSCEEEEEE...
3,1A3C.A,MNQKAVILDEQAIRRALTRIAHEMIERNKGMNNCILVGIKTRGIYL...,CCCEEEEECHHHHHHHHHHHHHHHHHHCCCCCCCEEEEESHHHHHH...
4,1A41.A,NAKRDRIFVRVYNVMKRINCFINKNIKKSSTDSNYQLAVFMLMETM...,CHHHHHHHHHHHHHHHHHHHHHHHHTTSTTTCTTHHHHHHHHHHHC...
5,1A5T.A,MRWYPWLRPDFEKLVASYQAGRGHHALLIQALPGMGDDALIYALSR...,CCCCGGGHHHHHHHHHHHHTTCCCSEEEEECCTTSCHHHHHHHHHH...
6,1A62.A,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSG...,CBHHHHHTSCHHHHHHHHHTTTCCCCTTSCHHHHHHHHHHHHHHTT...
7,1A73.A,MALTNAQILAVIDSWEETVGQFPVITHHVPLGGGLQGTLHCYEIPL...,CCCCHHHHHHHHHHHHHHHTTSCEEEEEEEEETTEEEEEEEEBCCS...
8,1A7J.A,MSKKHPIISVTGSSGAGTSTVKHTFDQIFRREGVKAVSIEGDAFHR...,CCTTSCEEEEESCCCCCCCTHHHHHHHHHHHHTCCEEEEEGGGGBS...
9,1A92.B,GREDILEQWVSGRKKLEELERDLRKLKKKIKKLEEDNPWLGNIKGI...,CHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCTTHHHHHHH...


### Calculate secondary structure percentage  

In [6]:
# helper functions to calculate secondary structure percentage  
helix_percent_calculator = lambda s: (s.count('5') + s.count('H') + s.count('G')) / len(s.replace('X',''))                                                   
sheet_percent_calculator = lambda s: (s.count('E') + s.count('B')) / len(s.replace('X',''))                                                                  
coil_percent_calculator = lambda s: (s.count('S') + s.count('T') + s.count('C')) / len(s.replace('X',''))                                                    
                                                                                                                                                             
# calculate each secondary structure percentage                                                                                                              
df['alpha'] = df.secondary_structure.apply(helix_percent_calculator)                                                                                         
df['beta'] = df.secondary_structure.apply(sheet_percent_calculator)                                                                                          
df['coil'] = df.secondary_structure.apply(coil_percent_calculator)                                                                                           

df.head(10)

,structureChainId,sequence,secondary_structure,alpha,beta,coil
0,12AS.A,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,CCCCHHHHHHHHHHHHHHHHHHHHHHHCEEECCCCSEEETTSSCSC...,0.345455,0.206061,0.448485
1,16VP.A,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,CCSCCCCCCCCHHHHHHHHHHHHTCTTHHHHHHHHHHCCCCCSTTS...,0.469945,0.046448,0.483607
2,1A1X.A,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,CCCCCCCCCCCSEEEEEETTEEEETTSCEEEEEEEECSSCEEEEEE...,0.037037,0.472222,0.490741
3,1A3C.A,MNQKAVILDEQAIRRALTRIAHEMIERNKGMNNCILVGIKTRGIYL...,CCCEEEEECHHHHHHHHHHHHHHHHHHCCCCCCCEEEEESHHHHHH...,0.292818,0.314917,0.392265
4,1A41.A,NAKRDRIFVRVYNVMKRINCFINKNIKKSSTDSNYQLAVFMLMETM...,CHHHHHHHHHHHHHHHHHHHHHHHHTTSTTTCTTHHHHHHHHHHHC...,0.568376,0.081197,0.350427
5,1A5T.A,MRWYPWLRPDFEKLVASYQAGRGHHALLIQALPGMGDDALIYALSR...,CCCCGGGHHHHHHHHHHHHTTCCCSEEEEECCTTSCHHHHHHHHHH...,0.538922,0.086826,0.374251
6,1A62.A,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSG...,CBHHHHHTSCHHHHHHHHHTTTCCCCTTSCHHHHHHHHHHHHHHTT...,0.284615,0.261538,0.453846
7,1A73.A,MALTNAQILAVIDSWEETVGQFPVITHHVPLGGGLQGTLHCYEIPL...,CCCCHHHHHHHHHHHHHHHTTSCEEEEEEEEETTEEEEEEEEBCCS...,0.184049,0.325153,0.490798
8,1A7J.A,MSKKHPIISVTGSSGAGTSTVKHTFDQIFRREGVKAVSIEGDAFHR...,CCTTSCEEEEESCCCCCCCTHHHHHHHHHHHHTCCEEEEEGGGGBS...,0.393103,0.186207,0.420690
9,1A92.B,GREDILEQWVSGRKKLEELERDLRKLKKKIKKLEEDNPWLGNIKGI...,CHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCTTHHHHHHH...,0.860000,0.000000,0.140000


### Label structure with protein foldtype

In [7]:
# helper funtion to get protein fold type                                                                       
def protein_fold_type(data, minThreshold, maxThreshold):
    '''
    Returns fold type with three major secondary structure class:
    "alpha", "beta", "alpha+beta", and "other" based upon the fraction of alpha/beta content.

    Attributes:
        data (DataFrame<Row>): input dataframe with alpha, beta composition
        minThreshold (float): below this threshold, the secondary structure is ignored
        maxThreshold (float): above this threshold, the secondary structure is ignored
    '''
    if data.alpha > maxThreshold and data.beta < minThreshold:                                
        return "alpha"                                                                        
    elif data.beta > maxThreshold and data.alpha < minThreshold:                              
        return "beta"                                                                         
    elif data.alpha > maxThreshold and data.beta > minThreshold:                              
        return "alpha+beta"                                                                   
    else:                                                                                     
        return "other"
    
df["foldType"] = df.apply(protein_fold_type, minThreshold = 0.05, maxThreshold = 0.15, axis=1)
df.head(10)

,structureChainId,sequence,secondary_structure,alpha,beta,coil,foldType
0,12AS.A,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,CCCCHHHHHHHHHHHHHHHHHHHHHHHCEEECCCCSEEETTSSCSC...,0.345455,0.206061,0.448485,alpha+beta
1,16VP.A,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,CCSCCCCCCCCHHHHHHHHHHHHTCTTHHHHHHHHHHCCCCCSTTS...,0.469945,0.046448,0.483607,alpha
2,1A1X.A,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,CCCCCCCCCCCSEEEEEETTEEEETTSCEEEEEEEECSSCEEEEEE...,0.037037,0.472222,0.490741,beta
3,1A3C.A,MNQKAVILDEQAIRRALTRIAHEMIERNKGMNNCILVGIKTRGIYL...,CCCEEEEECHHHHHHHHHHHHHHHHHHCCCCCCCEEEEESHHHHHH...,0.292818,0.314917,0.392265,alpha+beta
4,1A41.A,NAKRDRIFVRVYNVMKRINCFINKNIKKSSTDSNYQLAVFMLMETM...,CHHHHHHHHHHHHHHHHHHHHHHHHTTSTTTCTTHHHHHHHHHHHC...,0.568376,0.081197,0.350427,alpha+beta
5,1A5T.A,MRWYPWLRPDFEKLVASYQAGRGHHALLIQALPGMGDDALIYALSR...,CCCCGGGHHHHHHHHHHHHTTCCCSEEEEECCTTSCHHHHHHHHHH...,0.538922,0.086826,0.374251,alpha+beta
6,1A62.A,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSG...,CBHHHHHTSCHHHHHHHHHTTTCCCCTTSCHHHHHHHHHHHHHHTT...,0.284615,0.261538,0.453846,alpha+beta
7,1A73.A,MALTNAQILAVIDSWEETVGQFPVITHHVPLGGGLQGTLHCYEIPL...,CCCCHHHHHHHHHHHHHHHTTSCEEEEEEEEETTEEEEEEEEBCCS...,0.184049,0.325153,0.490798,alpha+beta
8,1A7J.A,MSKKHPIISVTGSSGAGTSTVKHTFDQIFRREGVKAVSIEGDAFHR...,CCTTSCEEEEESCCCCCCCTHHHHHHHHHHHHTCCEEEEEGGGGBS...,0.393103,0.186207,0.420690,alpha+beta
9,1A92.B,GREDILEQWVSGRKKLEELERDLRKLKKKIKKLEEDNPWLGNIKGI...,CHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCTTHHHHHHH...,0.860000,0.000000,0.140000,alpha


### Split sequence into bi-grams

In [8]:
def ngrammer(s,n):
    '''helper funtion to split sequence into n-grams
    
    Attributes:
       s (string): sequence of amino acid
       n (int): n-gram
    '''
    ngram = []                                                                                
    i = 0                                                                                     
    if len(s) < 1:                                                                            
        return []                                                                             
    while i < len(s) - n + 1:                                                                 
        ngram.append(s[i: i + n])                                                             
        i += 1                                                                                
    return ngram

df['ngram'] = df.sequence.apply(ngrammer, n=2)
df.head(3)

,structureChainId,sequence,secondary_structure,alpha,beta,coil,foldType,ngram
0,12AS.A,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,CCCCHHHHHHHHHHHHHHHHHHHHHHHCEEECCCCSEEETTSSCSC...,0.345455,0.206061,0.448485,alpha+beta,"[MK, KT, TA, AY, YI, IA, AK, KQ, QR, RQ, QI, I..."
1,16VP.A,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,CCSCCCCCCCCHHHHHHHHHHHHTCTTHHHHHHHHHHCCCCCSTTS...,0.469945,0.046448,0.483607,alpha,"[SR, RM, MP, PS, SP, PP, PM, MP, PV, VP, PP, P..."
2,1A1X.A,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,CCCCCCCCCCCSEEEEEETTEEEETTSCEEEEEEEECSSCEEEEEE...,0.037037,0.472222,0.490741,beta,"[GS, SA, AG, GE, ED, DV, VG, GA, AP, PP, PD, D..."


### Convert sequence to word2vec

In [13]:
# get all ngrams
ngrams = list(df.ngram)

# create word2vec model
model = Word2Vec(ngrams, size=50, window=25, min_count=1)

# train word2vec model
model.train(ngrams, total_examples=len(ngrams), epochs=10)

# create vector features for each structureChainId
df['features'] = df.ngram.apply(lambda ng: np.mean(np.array([np.array(model.wv[n]) for n in ng]), axis=0)) 

df.head(3)

,structureChainId,sequence,secondary_structure,alpha,beta,coil,foldType,ngram,features
0,12AS.A,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,CCCCHHHHHHHHHHHHHHHHHHHHHHHCEEECCCCSEEETTSSCSC...,0.345455,0.206061,0.448485,alpha+beta,"[MK, KT, TA, AY, YI, IA, AK, KQ, QR, RQ, QI, I...","[0.07492238, -0.29434553, 0.17545463, -0.15837..."
1,16VP.A,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,CCSCCCCCCCCHHHHHHHHHHHHTCTTHHHHHHHHHHCCCCCSTTS...,0.469945,0.046448,0.483607,alpha,"[SR, RM, MP, PS, SP, PP, PM, MP, PV, VP, PP, P...","[0.18599406, -0.9524737, 0.36988583, -0.513729..."
2,1A1X.A,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,CCCCCCCCCCCSEEEEEETTEEEETTSCEEEEEEEECSSCEEEEEE...,0.037037,0.472222,0.490741,beta,"[GS, SA, AG, GE, ED, DV, VG, GA, AP, PP, PD, D...","[0.4235544, -0.5381708, 0.35269824, -0.6758413..."


### Save DataFrame

In [15]:
# save dataframe as picke
df.to_pickle("./input_features.pkl")